In [1]:
import pandas as pd
import dgl
import torch
from itertools import combinations
from sklearn.metrics import jaccard_score
import os
from tqdm import tqdm

os.environ['DGLBACKEND'] = 'pytorch'

def create_heterogeneous_graph(df_ratings, threshold=0.7):
    # 读取数据
    user_movie_matrix = df_ratings.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

    # 建立用户与用户的相似度
    user_pairs = combinations(user_movie_matrix.index, 2)
    user_similarity = [(u1, u2, {'weight': jaccard_score(user_movie_matrix.loc[u1], user_movie_matrix.loc[u2])})
                       for u1, u2 in tqdm(user_pairs, total=(len(user_movie_matrix.index)*(len(user_movie_matrix.index)-1))//2, desc="Calculating user similarities") if jaccard_score(user_movie_matrix.loc[u1], user_movie_matrix.loc[u2]) >= Uthreshold]

    # 建立电影与电影的相似度
    movie_pairs = combinations(user_movie_matrix.columns, 2)
    movie_similarity = [(m1, m2, {'weight': jaccard_score(user_movie_matrix[m1], user_movie_matrix[m2])})
                        for m1, m2 in tqdm(movie_pairs, total=(len(user_movie_matrix.columns)*(len(user_movie_matrix.columns)-1))//2, desc="Calculating movie similarities") if jaccard_score(user_movie_matrix[m1], user_movie_matrix[m2]) >= threshold]

    # 用户与电影的评分关系
    user_movie_interactions = [(row['userId'], row['movieId'], {'rating': row['rating']})
                               for index, row in df_ratings.iterrows()]

    # 建立异质图
    g = dgl.heterograph({
        ('user', 'similar', 'user'): user_similarity,
        ('movie', 'similar', 'movie'): movie_similarity,
        ('user', 'rates', 'movie'): user_movie_interactions
    })

    return g

# 载入数据
df_ratings = pd.read_csv('D:\CODE\multi-model knowledge graph multi-graph recommendation system\data\cleanuser_rating.csv')

# 创建异质图
hetero_graph = create_heterogeneous_graph(df_ratings)
print(hetero_graph)
dgl.save_graphs('hetero_graph.bin', [hetero_graph])

# 印出图的基本信息
print(hetero_graph.ntypes, hetero_graph.etypes)


ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [5]:
import pandas as pd
import dgl
import torch
from itertools import combinations
from sklearn.metrics import jaccard_score
import os
from tqdm import tqdm
import pickle
os.environ['DGLBACKEND'] = 'pytorch'

def create_heterogeneous_graph(df_ratings, Uthreshold=0.5,threshold=0.7, num_users=10):
    # 读取数据，并限制用户数量
    df_ratings = df_ratings[df_ratings['userId'].isin(df_ratings['userId'].unique()[:num_users])]
    user_movie_matrix = df_ratings.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

    # 建立用户与用户的相似度
    user_pairs = combinations(user_movie_matrix.index, 2)
    user_similarity = [((u1, u2), jaccard_score(user_movie_matrix.loc[u1], user_movie_matrix.loc[u2]))
                       for u1, u2 in tqdm(user_pairs, total=(len(user_movie_matrix.index)*(len(user_movie_matrix.index)-1))//2, desc="Calculating user similarities") if jaccard_score(user_movie_matrix.loc[u1], user_movie_matrix.loc[u2]) >= Uthreshold]

    # 建立电影与电影的相似度
    movie_pairs = combinations(user_movie_matrix.columns, 2)
    movie_similarity = [((m1, m2), jaccard_score(user_movie_matrix[m1], user_movie_matrix[m2]))
                        for m1, m2 in tqdm(movie_pairs, total=(len(user_movie_matrix.columns)*(len(user_movie_matrix.columns)-1))//2, desc="Calculating movie similarities") if jaccard_score(user_movie_matrix[m1], user_movie_matrix[m2]) >= threshold]

    # 用户与电影的评分关系
    user_movie_interactions = [(row['userId'], row['movieId'])
                               for index, row in df_ratings.iterrows()]

    # 建立异质图
    g = dgl.heterograph({
        ('user', 'similar', 'user'): [pair for pair, _ in user_similarity],
        ('movie', 'similar', 'movie'): [pair for pair, _ in movie_similarity],
        ('user', 'rates', 'movie'): user_movie_interactions
    },
    num_nodes_dict={
        'user': user_movie_matrix.index.max() + 1,
        'movie': user_movie_matrix.columns.max() + 1
    })

    return g


df_ratings = pd.read_csv('D:\CODE\multi-model knowledge graph multi-graph recommendation system\data\cleanuser_rating.csv')


hetero_graph = create_heterogeneous_graph(df_ratings)
print(hetero_graph)
#dgl.save_graphs('hetero_graph.bin', [hetero_graph])

# 保存图到 .pkl 文件
with open('hetero_graph.pkl', 'wb') as f:
    pickle.dump(hetero_graph, f)

print(hetero_graph.ntypes, hetero_graph.etypes)


Calculating movie similarities: 100%|██████████| 29403/29403 [00:49<00:00, 593.00it/s]


Graph(num_nodes={'movie': 125917, 'user': 137},
      num_edges={('movie', 'similar', 'movie'): 2173, ('user', 'rates', 'movie'): 388, ('user', 'similar', 'user'): 0},
      metagraph=[('movie', 'movie', 'similar'), ('user', 'movie', 'rates'), ('user', 'user', 'similar')])
['movie', 'user'] ['similar', 'rates', 'similar']


改進第一次創建圖ID不正確問題


In [10]:
import pandas as pd
import dgl
import torch
from itertools import combinations
from sklearn.metrics import jaccard_score
import os
from tqdm import tqdm
import pickle
os.environ['DGLBACKEND'] = 'pytorch'

def create_heterogeneous_graph(df_ratings, Uthreshold=0.5, threshold=0.7, num_users=10):
    # 限制到前10个用户
    top_user_ids = df_ratings['userId'].unique()[:num_users]
    df_ratings = df_ratings[df_ratings['userId'].isin(top_user_ids)]
    
    user_ids = df_ratings['userId'].unique()
    movie_ids = df_ratings['movieId'].unique()
    
    user_id_map = {uid: i for i, uid in enumerate(user_ids)}
    movie_id_map = {mid: i for i, mid in enumerate(movie_ids)}
    
    # 用户与电影的交互矩阵，为了相似度计算
    user_movie_matrix = df_ratings.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
    
    # 用户与用户的相似度
    user_pairs = combinations(user_ids, 2)
    user_similarity = [
        ((user_id_map[u1], user_id_map[u2]), jaccard_score(user_movie_matrix.loc[u1].fillna(0), user_movie_matrix.loc[u2].fillna(0)))
        for u1, u2 in user_pairs 
        if jaccard_score(user_movie_matrix.loc[u1].fillna(0), user_movie_matrix.loc[u2].fillna(0)) >= Uthreshold
    ]
    
    # 电影与电影的相似度
    movie_pairs = combinations(movie_ids, 2)
    movie_similarity = [
        ((movie_id_map[m1], movie_id_map[m2]), jaccard_score(user_movie_matrix[m1].fillna(0), user_movie_matrix[m2].fillna(0)))
        for m1, m2 in movie_pairs
        if jaccard_score(user_movie_matrix[m1].fillna(0), user_movie_matrix[m2].fillna(0)) >= threshold
    ]
    
    # 用户与电影的评分关系
    user_movie_interactions = [
        (user_id_map[row['userId']], movie_id_map[row['movieId']])
        for index, row in df_ratings.iterrows()
    ]
    
    # 建立异质图
    g = dgl.heterograph({
        ('user', 'similar', 'user'): [pair for pair, _ in user_similarity],
        ('movie', 'similar', 'movie'): [pair for pair, _ in movie_similarity],
        ('user', 'rates', 'movie'): user_movie_interactions
    },
    num_nodes_dict={
        'user': len(user_ids),
        'movie': len(movie_ids)
    })

    return g

df_ratings = pd.read_csv('D:\CODE\multi-model knowledge graph multi-graph recommendation system\data\cleanuser_rating.csv')


hetero_graph = create_heterogeneous_graph(df_ratings)
print(hetero_graph)
#dgl.save_graphs('hetero_graph.bin', [hetero_graph])

# 保存图到 .pkl 文件
with open('hetero_graph.pkl', 'wb') as f:
    pickle.dump(hetero_graph, f)

print(hetero_graph.ntypes, hetero_graph.etypes)

Graph(num_nodes={'movie': 243, 'user': 10},
      num_edges={('movie', 'similar', 'movie'): 2173, ('user', 'rates', 'movie'): 388, ('user', 'similar', 'user'): 0},
      metagraph=[('movie', 'movie', 'similar'), ('user', 'movie', 'rates'), ('user', 'user', 'similar')])
['movie', 'user'] ['similar', 'rates', 'similar']


新一版測試


In [3]:
import dgl
import torch
from itertools import combinations
from sklearn.metrics import jaccard_score
import pandas as pd
import pickle
def create_heterogeneous_graph(df_ratings, Uthreshold=0.5, threshold=0.7, num_users=10):
    # 限制到前10个用户
    top_user_ids = df_ratings['userId'].unique()[:num_users]
    df_ratings = df_ratings[df_ratings['userId'].isin(top_user_ids)]
    
    user_ids = df_ratings['userId'].unique()
    movie_ids = df_ratings['movieId'].unique()
    
    # 为用户和电影创建映射到连续整数的字典
    user_id_map = {uid: i for i, uid in enumerate(user_ids)}
    movie_id_map = {mid: i for i, mid in enumerate(movie_ids)}
    
    # 用户与电影的交互矩阵，为了相似度计算
    user_movie_matrix = df_ratings.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
    
    # 用户与用户的相似度
    user_similarity = [
        ((user_id_map[u1], user_id_map[u2]), jaccard_score(user_movie_matrix.loc[u1].fillna(0), user_movie_matrix.loc[u2].fillna(0)))
        for u1, u2 in combinations(user_ids, 2)
        if jaccard_score(user_movie_matrix.loc[u1].fillna(0), user_movie_matrix.loc[u2].fillna(0)) >= Uthreshold
    ]
    
    # 电影与电影的相似度
    movie_similarity = [
        ((movie_id_map[m1], movie_id_map[m2]), jaccard_score(user_movie_matrix[m1].fillna(0), user_movie_matrix[m2].fillna(0)))
        for m1, m2 in combinations(movie_ids, 2)
        if jaccard_score(user_movie_matrix[m1].fillna(0), user_movie_matrix[m2].fillna(0)) >= threshold
    ]
    
    # 用户与电影的评分关系
    user_movie_interactions = [
        (user_id_map[row['userId']], movie_id_map[row['movieId']])
        for index, row in df_ratings.iterrows()
    ]
    
    # 建立异质图
    g = dgl.heterograph({
        ('user', 'similar', 'user'): [pair for pair, _ in user_similarity],
        ('movie', 'similar', 'movie'): [pair for pair, _ in movie_similarity],
        ('user', 'rates', 'movie'): user_movie_interactions
    },
    num_nodes_dict={
        'user': len(user_ids),
        'movie': len(movie_ids)
    })
    
    # 添加节点数据，这里是正确的位置
    g.nodes['movie'].data['movie_id'] = torch.tensor(movie_ids)  # 使用实际的 movie_ids，而非映射后的索引

    return g
df_ratings = pd.read_csv('D:\CODE\multi-model knowledge graph multi-graph recommendation system\data\cleanuser_rating.csv')


hetero_graph = create_heterogeneous_graph(df_ratings)
print(hetero_graph)
#dgl.save_graphs('hetero_graph.bin', [hetero_graph])

# 保存图到 .pkl 文件
with open('hetero_graph.pkl', 'wb') as f:
    pickle.dump(hetero_graph, f)

print(hetero_graph.ntypes, hetero_graph.etypes)

Graph(num_nodes={'movie': 243, 'user': 10},
      num_edges={('movie', 'similar', 'movie'): 2173, ('user', 'rates', 'movie'): 388, ('user', 'similar', 'user'): 0},
      metagraph=[('movie', 'movie', 'similar'), ('user', 'movie', 'rates'), ('user', 'user', 'similar')])
['movie', 'user'] ['similar', 'rates', 'similar']


加入圖像 文字 聲音 實體

In [16]:
hetero_graph = create_heterogeneous_graph(df_ratings)
print(hetero_graph)
print("Node types:", hetero_graph.ntypes)
print("Edge types:", hetero_graph.etypes)

Graph(num_nodes={'movie': 243, 'user': 10},
      num_edges={('movie', 'similar', 'movie'): 2173, ('user', 'rates', 'movie'): 388, ('user', 'similar', 'user'): 0},
      metagraph=[('movie', 'movie', 'similar'), ('user', 'movie', 'rates'), ('user', 'user', 'similar')])
Node types: ['movie', 'user']
Edge types: ['similar', 'rates', 'similar']


In [4]:
import pandas as pd

def load_youtube_to_movie_mapping(filepath):
    df = pd.read_csv(filepath)
    # 创建从 movieId 到 youtubeId 的字典
    movie_to_youtube = pd.Series(df.youtubeId.values, index=df.movieId).to_dict()
    return movie_to_youtube

# 加载映射
movie_to_youtube = load_youtube_to_movie_mapping('D:\\CODE\\multi-model knowledge graph multi-graph recommendation system\\data\\ml-youtube_cleaned.csv')


影像特徵提取

In [ ]:
from moviepy.editor import VideoFileClip
import torchvision.transforms as transforms
import torchvision.models as models
import torch
from PIL import Image
import numpy as np
import os
from tqdm import tqdm

# 加载预训练的 ResNet50
resnet50 = models.resnet50(pretrained=True)
resnet50.eval()
resnet50 = resnet50.to('cuda')

def process_frame(img):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img = Image.fromarray(img)
    img_t = preprocess(img).unsqueeze(0).to('cuda')
    with torch.no_grad():
        features = resnet50(img_t)
    return features.cpu().numpy()

def extract_features_from_video(video_path):
    try:
        clip = VideoFileClip(video_path).subclip(0, 60)  # 处理前三分钟
        frame_features = [process_frame(frame) for frame in tqdm(clip.iter_frames(fps=1), total=180, desc=f"Processing {os.path.basename(video_path)}")]
        clip.close()  # 明确关闭clip释放资源
        return np.mean(frame_features, axis=0)
    except Exception as e:
        print(f"Error processing video {video_path}: {str(e)}")
        return None

def extract_features_for_movies(graph, videos_folder, movie_to_youtube):
    features_list = []  # 用来存储所有电影的特征
    movie_indices = []  # 用来记录有特征的电影索引

    for idx, movie_id in enumerate(tqdm(graph.nodes['movie'].data['movie_id'], desc="Extracting features for movies")):
        youtube_id = movie_to_youtube.get(movie_id.item())  # 保证使用正确的 movie_id
        if youtube_id:
            video_path = os.path.join(videos_folder, f"{youtube_id}.mp4")
            if os.path.exists(video_path):
                features = extract_features_from_video(video_path)
                if features is not None:
                    features_list.append(features)
                    movie_indices.append(idx)  # 记录成功获取特征的电影索引
                else:
                    print(f"Failed to extract features for {video_path}")
            else:
                print(f"Video for youtubeId {youtube_id} not found")
        else:
            print(f"No youtubeId found for movieId {movie_id}")

    if features_list:
        # 将所有特征合并为一个张量，并存储到图中
       
        # 提取特征并在存储前调整形状
        features_tensor = torch.stack([torch.tensor(feat).squeeze() for feat in features_list])  # 确保每个特征是二维的
        # 为了安全起见，再次确认形状
        expected_shape = (len(graph.nodes['movie'].data['movie_id']), features_tensor.shape[1])
        graph.nodes['movie'].data['features'] = torch.zeros(expected_shape)
        graph.nodes['movie'].data['features'][movie_indices] = features_tensor

# 调用函数
extract_features_for_movies(hetero_graph, 'D:\\CODE\\multi-model knowledge graph multi-graph recommendation system\\data\\videos', movie_to_youtube)


In [5]:
import torch
from tqdm import tqdm
import os
import pandas as pd
import dgl
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from moviepy.editor import VideoFileClip
import numpy as np

def extract_features_for_movies(graph, videos_folder, movie_to_youtube):
    movie_ids = graph.nodes['movie'].data['movie_id'].numpy()  # 获取图中所有电影ID
    num_movies = len(movie_ids)
    
    # 为影像特征创建节点
    graph.add_nodes(num_movies, ntype='movieimage')
    features_list = []

    for idx, movie_id in enumerate(tqdm(movie_ids, desc="Extracting features for movies")):
        youtube_id = movie_to_youtube.get(movie_id)
        if youtube_id:
            video_path = os.path.join(videos_folder, f"{youtube_id}.mp4")
            if os.path.exists(video_path):
                features = extract_features_from_video(video_path)
                if features is not None:
                    features_list.append(torch.tensor(features))
                    graph.add_edge(movie_id, idx, etype=('movie', 'has_image', 'movieimage'))
                else:
                    print(f"Failed to extract features for {video_path}")
            else:
                print(f"Video for youtubeId {youtube_id} not found")
        else:
            print(f"No youtubeId found for movieId {movie_id}")
            features_list.append(torch.zeros(2048))  # Assuming features are 2048-dimensional

    # 将所有特征合并为一个张量并存储
    if features_list:
        features_tensor = torch.stack(features_list)
        graph.nodes['movieimage'].data['features'] = features_tensor

# 实例化模型
resnet50 = models.resnet50(pretrained=True)
resnet50.eval()
resnet50 = resnet50.to('cuda')

# 调用函数
extract_features_for_movies(hetero_graph, 'D:\\CODE\\multi-model knowledge graph multi-graph recommendation system\\data\\videos', movie_to_youtube)

# 保存图到新的.pkl文件
with open('hetero_graph_with_images.pkl', 'wb') as f:
    pickle.dump(hetero_graph, f)


DGLError: Node type "movieimage" does not exist.

檢查是否正確加到圖中

In [28]:
# 检查 'movie' 节点是否包含 'features' 数据
if 'features' in hetero_graph.nodes['movie'].data:
    print("Features are available in the graph.")
else:
    print("Features are not set in the graph.")
# 如果特征存在，打印出特征的形状
if 'features' in hetero_graph.nodes['movie'].data:
    features_shape = hetero_graph.nodes['movie'].data['features'].shape
    print("Shape of features:", features_shape)
# 查看前几个电影节点的特征
if 'features' in hetero_graph.nodes['movie'].data:
    sample_features = hetero_graph.nodes['movie'].data['features'][:5]
    print("Sample features:\n", sample_features)


Features are available in the graph.
Shape of features: torch.Size([243, 1000])
Sample features:
 tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.7366,  0.4332, -0.7367,  ..., -2.0172,  0.8428,  0.0202],
        [-0.8027,  0.6168, -0.4021,  ..., -1.6636,  0.4474,  0.6568],
        [-0.8751,  0.6143, -0.0608,  ..., -1.6497,  0.2464,  0.2390],
        [ 0.4690,  0.1685, -0.1857,  ..., -1.7945,  0.8671,  0.9250]])


存成pkl

In [29]:
import pickle
# 路径可以根据你的具体需求进行更改
file_path = 'D:\\CODE\\multi-model knowledge graph multi-graph recommendation system\\data\\hetero_graph_with_features.pkl'

# 打开一个文件以写入二进制模式，并保存图
with open(file_path, 'wb') as f:
    pickle.dump(hetero_graph, f)


In [30]:
import pickle
import dgl
# 替换成你保存 .pkl 文件的实际路径
file_path = 'D:\\CODE\\multi-model knowledge graph multi-graph recommendation system\\data\\hetero_graph_with_features.pkl'

# 从文件中加载图
with open(file_path, 'rb') as f:
    loaded_graph = pickle.load(f)
print("Loaded graph info:")
print(loaded_graph)
print("Node types:", loaded_graph.ntypes)
print("Edge types:", loaded_graph.etypes)
# 检查 'movie' 节点的特征数据
if 'features' in loaded_graph.nodes['movie'].data:
    print("Features of 'movie' nodes are available.")
    print("Sample features from 'movie' nodes:", loaded_graph.nodes['movie'].data['features'][:5])
else:
    print("Features data is not available in 'movie' nodes.")


Loaded graph info:
Graph(num_nodes={'movie': 243, 'user': 10},
      num_edges={('movie', 'similar', 'movie'): 2173, ('user', 'rates', 'movie'): 388, ('user', 'similar', 'user'): 0},
      metagraph=[('movie', 'movie', 'similar'), ('user', 'movie', 'rates'), ('user', 'user', 'similar')])
Node types: ['movie', 'user']
Edge types: ['similar', 'rates', 'similar']
Features of 'movie' nodes are available.
Sample features from 'movie' nodes: tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.7366,  0.4332, -0.7367,  ..., -2.0172,  0.8428,  0.0202],
        [-0.8027,  0.6168, -0.4021,  ..., -1.6636,  0.4474,  0.6568],
        [-0.8751,  0.6143, -0.0608,  ..., -1.6497,  0.2464,  0.2390],
        [ 0.4690,  0.1685, -0.1857,  ..., -1.7945,  0.8671,  0.9250]])


In [12]:
print("节点类型:", hetero_graph.ntypes)
print("边类型:", hetero_graph.etypes)
print("节点数量:", {ntype: hetero_graph.num_nodes(ntype) for ntype in hetero_graph.ntypes})
# 打印每种边类型的数量
# 确保使用完整的边类型名称，形如 ('node_type1', 'edge_type', 'node_type2')
print("边数量:", {etype: hetero_graph.num_edges(etype) for etype in hetero_graph.canonical_etypes})
# 打印元图结构
print("元图结构:", hetero_graph.metagraph().edges())



节点类型: ['movie', 'user']
边类型: ['similar', 'rates', 'similar']
节点数量: {'movie': 243, 'user': 10}
边数量: {('movie', 'similar', 'movie'): 2173, ('user', 'rates', 'movie'): 388, ('user', 'similar', 'user'): 0}
元图结构: [('movie', 'movie'), ('user', 'movie'), ('user', 'user')]


In [31]:
# 检查 'movie' 节点是否包含 'features' 数据
if 'features' in loaded_graph.nodes['movie'].data:
    print("Features are stored in the graph under 'movie' nodes.")
else:
    print("No features data found in 'movie' nodes.")
if 'features' in loaded_graph.nodes['movie'].data:
    features = loaded_graph.nodes['movie'].data['features']
    print("Shape of features:", features.shape)
    print("Type of features:", features.dtype)
    # 打印一些统计信息，如最大值、最小值等
    print("Feature stats - max:", torch.max(features, dim=0)[0], "min:", torch.min(features, dim=0)[0])
if 'features' in loaded_graph.nodes['movie'].data:
    # 查看前五个电影节点的特征
    print("Sample features from the first five 'movie' nodes:")
    print(loaded_graph.nodes['movie'].data['features'][:5])


Features are stored in the graph under 'movie' nodes.
Shape of features: torch.Size([243, 1000])
Type of features: torch.float32
Feature stats - max: tensor([3.2356e+00, 1.7401e+00, 2.0369e+00, 2.2780e+00, 3.3385e+00, 3.3502e+00,
        9.9514e-01, 5.0645e-01, 0.0000e+00, 1.3633e+00, 2.9104e-01, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 8.2428e-02, 0.0000e+00, 5.7014e-01,
        0.0000e+00, 0.0000e+00, 1.0420e-01, 1.4161e+00, 1.1383e+00, 1.7588e+00,
        2.7759e-01, 9.7179e-01, 1.7401e+00, 9.5744e-01, 2.2160e-01, 1.9017e+00,
        1.2646e+00, 6.5758e-01, 1.3739e+00, 1.8477e+00, 2.7144e+00, 1.2448e+00,
        2.2278e+00, 1.8292e-01, 1.9878e+00, 2.2261e+00, 0.0000e+00, 1.3953e-01,
        3.4531e-01, 1.5675e+00, 4.9841e-01, 1.6277e+00, 2.7734e-01, 3.0778e+00,
        9.2187e-01, 1.6390e+00, 1.6008e+00, 2.9234e+00, 5.5483e-01, 7.8193e-03,
        7.3927e-01, 4.9543e-01, 0.0000e+00, 0.0000e+00, 1.6650e+00, 2.2373e+00,
        1.5630e+00, 9.4276e-01, 1.6401e+00, 2.5345

In [13]:
# 检查 'user' 到 'user' 的 'similar' 边权重
if ('user', 'similar', 'user') in hetero_graph.etypes:
    if 'weight' in hetero_graph.edges[('user', 'similar', 'user')].data:
        print("一些用户间相似边的权重:", hetero_graph.edges[('user', 'similar', 'user')].data['weight'][:5])

# 检查 'movie' 到 'movie' 的 'similar' 边权重
if ('movie', 'similar', 'movie') in hetero_graph.etypes:
    if 'weight' in hetero_graph.edges[('movie', 'similar', 'movie')].data:
        print("一些电影间相似边的权重:", hetero_graph.edges[('movie', 'similar', 'movie')].data['weight'][:5])


In [14]:
# 检查 'user' 到 'user' 的 'similar' 边数量
num_user_similar_edges = hetero_graph.num_edges(('user', 'similar', 'user'))
print("用户间相似边的数量:", num_user_similar_edges)

# 检查 'movie' 到 'movie' 的 'similar' 边数量
num_movie_similar_edges = hetero_graph.num_edges(('movie', 'similar', 'movie'))
print("电影间相似边的数量:", num_movie_similar_edges)


用户间相似边的数量: 0
电影间相似边的数量: 2173


In [ ]:
import dgl
import torch

# 创建一个简单的图
src = torch.tensor([0, 1, 2])
dst = torch.tensor([1, 2, 3])
g = dgl.graph((src, dst))

# 将图转移到 GPU
g = g.to('cuda')

# 打印图的设备信息来确认其在 GPU 上
print('图数据位于:', g.device)

# 如果你需要使用特征，也确保它们在 GPU 上
features = torch.randn(4, 3)  # 假设有 4 个节点，每个节点 3 个特征
features = features.to('cuda')
g.ndata['feat'] = features

# 检查特征是否已移至 GPU
print('节点特征位于:', g.ndata['feat'].device)


In [ ]:
import torch
import dgl

print(torch.cuda.is_available())  # 应返回 True
print(dgl.__path__)
